In [1]:
import pandas as pd
import numpy as np

In [2]:
orders_leden = pd.read_excel ('sample_data.xlsx', skiprows=1)

In [3]:
orders_leden.dtypes

#0 registered id              object
#1 timestamp                   int64
#2 transactiontype            object
#3 productname                object
#4 HW                         object
count(order_thank_you_gtm)     int64
dtype: object

In [4]:
orders_leden

,#0 registered id,#1 timestamp,#2 transactiontype,#3 productname,#4 HW,count(order_thank_you_gtm)
0,"[u'0000DEC299', u'58E2172406', u'12775E0E79', ...",1517180400,[u’GSM_aa'],[u’Mobil'],[u’Android'],1
1,"[u'004435F113', u'5893734697', u'6AFF218A62', ...",1515538800,[u’GSM_aa'],[u’Mobil'],[u’Android'],1
2,"[u'017DFBCC41', u'E3F8D70EB5', u'793EFF2FE9', ...",1516662000,[u’GSM_aa'],[u’Mobil'],[u’Android'],1
3,"[u'01C3664100', u'9A11EE47BE', u'CEF3E6FF8F', ...",1515625200,[u’GSM_aa'],[u’Mobil'],[u’Android'],1
4,"[u'016227BE25', u'52ED9ADB4B', u'A7BEF2D833', ...",1516834800,[u’GSM_aa'],[u’Mobil'],[u’Android'],1
5,"[u'02365360FB', u'02511FA97C', u'34020989F7', ...",1515711600,[u’GSM_aa'],[u’Mobil'],[u’Android'],1
6,"[u'029B32F15C', u'B2D0FE5B04', u'E75B68432F', ...",1516143600,[u’GSM_aa'],[u’Mobil'],[u’Android'],1
7,"[u'023B34F12C', u'B7D9FE4B03', u'E62B64352F', ...",1517094000,[u’GSM_aa'],[u’Mobil'],[u’Android'],1
8,"[u'02786DDDF2', u'2440797832', u'372BAB366C', ...",1514847600,[u’GSM_aa'],[u’Mobil'],[u’Android'],1
9,"[u'025E6D9E81', u'0C1D5B8913', u'D38C2C424C', ...",1515020400,[u’GSM_aa'],[u’Mobil'],[u’Android'],1


V datech nahoře je dobře videt, že ten sloupec `#0 registered id`je trochu zprasený: je to text, který obsahuje jedno z:
* něco, co vypadá jako seznam (`[...]`), kdy jednotlivé položky jsou idčka jako řetězce ve speciální Python notaci (to je to `u'abcdec'`, všimněte si `u` a `'` uvozovek) - odhaduju, že do toho Excelu to někdo vysypal v Pythonu
* osamocené idčko, které už ale nevypadá jako seznam a ani jako Python řetězec

Taky jsou v některých případech špatně uvozovky, namísto znaku `'` (jednoduchá uvozovka) je tam `’`.

Abychom mohli rozdělit podle hodnot ve sloupečku `#0 registered id` ve vícero řádků, potřebujeme to mít ve sloupečku, kde bude opravdový Python seznam, nikoliv text, který jako seznam vypadá. Na to použiju `apply`:
* Dodám funkci, kterou jsem si pojmenoval `list_string_to_list`. pandas ji zavolá pro každý řádek a úkolem té mojí funkce je vrátit hodnotu buňky. V našem případě to bude vždycky seznam.
* V té funkci ošetřuji různé případy, které jsem popsal nahoře.
* Volám `apply`.
* Nakonec vytvořím nový sloupec, kde bude to "nové id" (`new_id`), abych později poznal, které řádky patří k sobě (tj. byly vytvořeny ze stejného řádku v původní tabulce).

In [5]:
import ast
def list_string_to_list(row):
    text = row["#0 registered id"]
    text = text.replace("’", "'")
    if text.startswith("["):   
        id_list = ast.literal_eval(text)
        return id_list
    else:
        return [text]

orders_leden["#0 registered id"] = orders_leden.apply(list_string_to_list, axis=1)

orders_leden["new_id"] = orders_leden.index

In [6]:
orders_leden

,#0 registered id,#1 timestamp,#2 transactiontype,#3 productname,#4 HW,count(order_thank_you_gtm),new_id
0,"[0000DEC299, 58E2172406, 12775E0E79, 01D310D17...",1517180400,[u’GSM_aa'],[u’Mobil'],[u’Android'],1,0
1,"[004435F113, 5893734697, 6AFF218A62, C7EA682E1...",1515538800,[u’GSM_aa'],[u’Mobil'],[u’Android'],1,1
2,"[017DFBCC41, E3F8D70EB5, 793EFF2FE9, D3F011A35...",1516662000,[u’GSM_aa'],[u’Mobil'],[u’Android'],1,2
3,"[01C3664100, 9A11EE47BE, CEF3E6FF8F, 82377BCF4...",1515625200,[u’GSM_aa'],[u’Mobil'],[u’Android'],1,3
4,"[016227BE25, 52ED9ADB4B, A7BEF2D833, AA854665B...",1516834800,[u’GSM_aa'],[u’Mobil'],[u’Android'],1,4
5,"[02365360FB, 02511FA97C, 34020989F7, 3FC627B5C...",1515711600,[u’GSM_aa'],[u’Mobil'],[u’Android'],1,5
6,"[029B32F15C, B2D0FE5B04, E75B68432F, A2A64D30B...",1516143600,[u’GSM_aa'],[u’Mobil'],[u’Android'],1,6
7,"[023B34F12C, B7D9FE4B03, E62B64352F, A1A66D70B...",1517094000,[u’GSM_aa'],[u’Mobil'],[u’Android'],1,7
8,"[02786DDDF2, 2440797832, 372BAB366C, 041AEAAB8...",1514847600,[u’GSM_aa'],[u’Mobil'],[u’Android'],1,8
9,"[025E6D9E81, 0C1D5B8913, D38C2C424C, 12CDD3FD4...",1515020400,[u’GSM_aa'],[u’Mobil'],[u’Android'],1,9


Teď budeme z každého jednoho řádku dělat vícero, na základě seznamu idček ve sloupci `#0 registered id`. Postup jsem normáka vykradl z https://stackoverflow.com/a/46983212. K tomu, aby člověk pochopil, jak přesně to funguje, musí rozumět každé z těch funkcí, které se pro to používají: `set_index`, `apply`, `stack` i `reset_index`. Je to dost magic a ne že ne :)

In [7]:
column_to_explode = '#0 registered id'
res = (orders_leden
       .set_index([x for x in orders_leden.columns if x != column_to_explode])[column_to_explode]
       .apply(pd.Series)
       .stack()
       .reset_index())
res

,#1 timestamp,#2 transactiontype,#3 productname,#4 HW,count(order_thank_you_gtm),new_id,level_6,0
0,1517180400,[u’GSM_aa'],[u’Mobil'],[u’Android'],1,0,0,0000DEC299
1,1517180400,[u’GSM_aa'],[u’Mobil'],[u’Android'],1,0,1,58E2172406
2,1517180400,[u’GSM_aa'],[u’Mobil'],[u’Android'],1,0,2,12775E0E79
3,1517180400,[u’GSM_aa'],[u’Mobil'],[u’Android'],1,0,3,01D310D170
4,1517180400,[u’GSM_aa'],[u’Mobil'],[u’Android'],1,0,4,244CB07C63
5,1517180400,[u’GSM_aa'],[u’Mobil'],[u’Android'],1,0,5,F8529FE096
6,1517180400,[u’GSM_aa'],[u’Mobil'],[u’Android'],1,0,6,469E924B73
7,1517180400,[u’GSM_aa'],[u’Mobil'],[u’Android'],1,0,7,25393149F3
8,1517180400,[u’GSM_aa'],[u’Mobil'],[u’Android'],1,0,8,A48F22F2FF
9,1517180400,[u’GSM_aa'],[u’Mobil'],[u’Android'],1,0,9,9662101934


In [21]:
res[(11 <= res["level_6"]) & (res["level_6"] >= 2) & (8 <= res["level_6"]) & (res["level_6"] >= 2)]

,#1 timestamp,#2 transactiontype,#3 productname,#4 HW,count(order_thank_you_gtm),new_id,level_6,0
22,1515538800,[u’GSM_aa'],[u’Mobil'],[u’Android'],1,1,11,49B3D5CC65
23,1515538800,[u’GSM_aa'],[u’Mobil'],[u’Android'],1,1,12,47D0B5761D
24,1515538800,[u’GSM_aa'],[u’Mobil'],[u’Android'],1,1,13,DD4DBBB899
25,1515538800,[u’GSM_aa'],[u’Mobil'],[u’Android'],1,1,14,14B8DDA486
26,1515538800,[u’GSM_aa'],[u’Mobil'],[u’Android'],1,1,15,3659C48AA1
27,1515538800,[u’GSM_aa'],[u’Mobil'],[u’Android'],1,1,16,473AA3D7D0
28,1515538800,[u’GSM_aa'],[u’Mobil'],[u’Android'],1,1,17,BAB73BD11C
29,1515538800,[u’GSM_aa'],[u’Mobil'],[u’Android'],1,1,18,07D55ABA37
30,1515538800,[u’GSM_aa'],[u’Mobil'],[u’Android'],1,1,19,826AA6B0C5
31,1515538800,[u’GSM_aa'],[u’Mobil'],[u’Android'],1,1,20,46C52E3FBC
